# Step1 - Finetuning

In the initial fine-tuning step (Step 1), our focus was on instilling a foundational comprehension of the topics of SHAP and batteries. The aim was to provide the model with a broad understanding of the specified subjects. The following notebook describes the different fine-tuning steps conducted. Thereby, the conda-kernel "textgen" is used.

## Start Training
To start the training, we have to restart the textgeneration-webui, using the following commands:

    conda activate textgen
    cd /home/[...]/text-generation-webui/
    python server.py --share --model models/Llama-2-13b-chat-hf --load-in-8bit

Thereby, we load the selected base model (Llama-2-13b-chat-hf) in 8bit as recommended by the text-generation-webui-documentation.

After the webui started, the following steps have been executed within the webui:

1. Switch to tab "Training"
2. Give the LoRA-file a new name - in our case "Step1_adapter"
3. Adapt the Hyperparameters for the training:
- Training-epochs: 3
- Learning-rate: 3e^-4
- LoRA Rank: 8
- LoRA Alpha: 16
- Batch Size: 128
- Micro Batch Size: 4
- Cutoff Length: 256
- LR Scheduler: linear
- Overlap Length: 128
- Prefer Newline Cut Length: 128
4. Go to the Tab "Raw text file"
5. Select under "Text file" the folder "Step1-Trainingdata"
6. Click "Start LoRA Training"

After some minutes the new LoRA-adapter was saved to the folder "text-generation-webui/loras/Step1_adapter".

## Create Intermediate-Step1-model
We decided to create the new Intermediate-Step1-model by merging the newly trained LORA-adapter (Step1_adapter) with the corresponding base model (Llama-2-13b-chat-hf). Therefore, we executed the following code from this notebook:

### Load Base-model, tokenizer, LoRA-adapter

In [20]:
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_path = "/home/jupyter-edt_wise34_tf2/text-generation-webui/models/Llama-2-13b-chat-hf/"


model = AutoModelForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="offload",
    trust_remote_code=True
)
model.config.use_cache = False

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### Combine Base-model with LoRA-adapter

In [28]:
peft_model = PeftModel.from_pretrained(model, "/home/jupyter-edt_wise34_tf2/text-generation-webui/loras/Step1_adapter/", is_trainable=True)

### Merge Base-model with LoRA-adapter 

In [24]:
merged_model = peft_model.merge_and_unload()

### Save merged model to text-generation-webui

In [ ]:
merged_model.save_pretrained("/home/jupyter-edt_wise34_tf2/text-generation-webui/models/Step1-model/", safe_serializetion=True)
tokenizer.save_pretrained("/home/jupyter-edt_wise34_tf2/text-generation-webui/models/Step1-model/")

The new Step1-model is saved to the folder "/home/[...]/text-generation-webui/models".